In [1]:
import pandas as pd
import pickle 
import json

In [2]:
def save_to_disk(data, filepath):
    with open(filepath, 'wb') as file:
        pickle.dump(data, file)
def load_from_disk(filepath):
    with open(filepath, 'rb') as file:
        data = pickle.load(file)
        return data

In [3]:
# Read Json 
json_path = "./AllBuildingList_TaskAssignment.json"
with open(json_path, 'r', encoding="utf-8") as json_file:
    buildings_dict = json.load(json_file)

In [4]:
# Extract building names 
all_building_names = []
for elem in buildings_dict:
    all_building_names.append(elem["BuildingName"])
print("All Building names (" + str(len(all_building_names)) + ", unique names " + str(len(list(set((all_building_names))))) + "): \n", all_building_names)

All Building names (284, unique names 247): 
 ['Building_92', 'Garage_224', 'Building_226', 'Building_58', 'Building_182', 'Building_101', 'Building_225', 'Building_105', 'Building_139', 'Building_161', 'Garage_185', 'Building_146', 'Building_229', 'Building_86', 'Building_216', 'Building_215', 'Building_175', 'Building_196', 'Building_228', 'Building_124', 'Building_115', 'Building_192', 'Building_174', 'Building_219', 'Building_213', 'Building_192', 'Building_116', 'Building_116', 'Building_115', 'Building_189', 'Building_207', 'Building_193', 'Building_158', 'Building_123', 'Building_154', 'Building_125', 'Building_206', 'Building_191', 'Building_212', 'Building_190', 'Building_151', 'Building_84', 'Building_122', 'Building_106', 'Garage_185', 'Building_198', 'Building_131', 'Building_179', 'Building_201', 'Building_165', 'Building_184', 'Building_171', 'Building_176', 'Building_177', 'Building_208', 'Building_185', 'Building_178', 'Building_121', 'Building_157', 'Building_164', 'Bu

In [5]:
# Filter buildings
exclude_parts = [] # ["garage"]
filtered_names = []
for elem in all_building_names:
    for exclude in exclude_parts:
        if exclude in elem.lower():
            filtered_names.append(elem)
filtered_names = list(set(all_building_names) - set(filtered_names))
print("Filtered names (" + str(len(filtered_names)) + "): \n", filtered_names)

Filtered names (247): 
 ['Building_172', 'Building_174', 'Building_230', 'Building_95', 'Building_66', 'Building_186', 'TaskBuilding_20', 'Building_138', 'Building_154', 'TaskBuilding_39', 'Building_179', 'Building_198', 'Building_233', 'TaskBuilding_36', 'Building_119', 'Building_204', 'Building_92', 'Building_197', 'Building_93', 'Building_112', 'TaskBuilding_30', 'Building_109', 'Building_202', 'Building_73', 'Building_113', 'Building_167', 'Building_156', 'TaskBuilding_8', 'TaskBuilding_11', 'Building_94', 'Building_70', 'Building_135', 'Building_175', 'Building_220', 'Building_169', 'Building_102', 'TaskBuilding_52', 'Building_114', 'Building_62', 'TaskBuilding_33', 'Building_196', 'Building_203', 'Building_79', 'Building_71', 'TaskBuilding_55', 'Garage_235', 'Building_105', 'Building_98', 'Building_88', 'Building_91', 'Building_162', 'Building_234', 'Building_218', 'Building_96', 'TaskBuilding_25', 'Building_191', 'Building_201', 'Building_124', 'Garage_98', 'Building_149', 'Task

In [6]:
# rename buildings
rename_dict = {'Garage_235':'Building_235', 'Garage_224':'Building_224', 'Garage_185':'Building_185', \
                'Garage_71':'Building_71', 'Garage_86':'Building_86', 'Garage_98':'Building_98', \
                'Garage_26':'TaskBuilding_26', 'Garage_46':'TaskBuilding_46', 'Garage_72':'Building_72'}

renamed_buildings = []
for elem in filtered_names:
    if elem in [*rename_dict]:
        renamed_buildings.append(rename_dict[elem])
    else:
        renamed_buildings.append(elem)
print("Renamed buildings (" + str(len(renamed_buildings)) + "): \n" + str(renamed_buildings))       



Renamed buildings (247): 
['Building_172', 'Building_174', 'Building_230', 'Building_95', 'Building_66', 'Building_186', 'TaskBuilding_20', 'Building_138', 'Building_154', 'TaskBuilding_39', 'Building_179', 'Building_198', 'Building_233', 'TaskBuilding_36', 'Building_119', 'Building_204', 'Building_92', 'Building_197', 'Building_93', 'Building_112', 'TaskBuilding_30', 'Building_109', 'Building_202', 'Building_73', 'Building_113', 'Building_167', 'Building_156', 'TaskBuilding_8', 'TaskBuilding_11', 'Building_94', 'Building_70', 'Building_135', 'Building_175', 'Building_220', 'Building_169', 'Building_102', 'TaskBuilding_52', 'Building_114', 'Building_62', 'TaskBuilding_33', 'Building_196', 'Building_203', 'Building_79', 'Building_71', 'TaskBuilding_55', 'Building_235', 'Building_105', 'Building_98', 'Building_88', 'Building_91', 'Building_162', 'Building_234', 'Building_218', 'Building_96', 'TaskBuilding_25', 'Building_191', 'Building_201', 'Building_124', 'Building_98', 'Building_149',

In [7]:
# save data
renamed_buildings.sort()
save_to_disk(renamed_buildings,"./building_names.pickle")